## Sistema de recomendacion ML 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn .feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

Metodo a usar : Similitud de coseno.

In [2]:
df_movies  = pd.read_csv('../data/df_movies_limpio.csv')

# max_df=0.8, min_df=5, max_features=1000 = 1m 6.2s
# max_df=0.7, min_df=5, max_features=1000 = 1m 12.8s
# max_df=0.7, min_df=10, max_features=1000 = 58.9s


In [18]:
vectorizer = TfidfVectorizer(max_df=0.8, min_df=10, max_features=200)
tfidf_matrix = vectorizer.fit_transform(df_movies['title'])

svd = TruncatedSVD(n_components=100)  
tfidf_reduced = svd.fit_transform(tfidf_matrix)

company_vectorizer = TfidfVectorizer(max_df=0.8, min_df=10, max_features=200)
company_tfidf_matrix = company_vectorizer.fit_transform(df_movies['company_name'])

# Reducir la dimensionalidad de la matriz TF-IDF de 'company_name'
svd_company = TruncatedSVD(n_components=100)
company_tfidf_reduced = svd_company.fit_transform(company_tfidf_matrix)

features = np.column_stack([tfidf_reduced, company_tfidf_reduced, df_movies['popularity'].values])
similarity_matrix = cosine_similarity(features)

In [22]:
nombre_pelicula = "dracula: dead and loving it"  
pelicula = df_movies[df_movies['title'] == nombre_pelicula]

if not pelicula.empty:
    pelicula_index = pelicula.index[0]
    pelicula_similarities = similarity_matrix[pelicula_index]
    most_similar_pelicula_indices = np.argsort(-pelicula_similarities)

    # Excluir el índice del producto en sí mismo
    most_similar_products_indices = most_similar_pelicula_indices[most_similar_pelicula_indices != pelicula_index]

    # Obtenemos los nombres de los 5 productos más similares, excluyendo el producto en sí mismo
    top_5_similar_products = df_movies.loc[most_similar_products_indices[:5], 'title']
    print("Las 5 películas más similares a",nombre_pelicula, "son:")
    print(top_5_similar_products)
else:
    print("Producto no encontrado")


Las 5 películas más similares a dracula: dead and loving it son:
4838      dead reckoning
13039      jack and jill
11186         this is it
12473    just go with it
1145         double team
Name: title, dtype: object
